# MINI-SYNTH-GUI

ready to rock mini synth GUI

In [ ]:
import sys
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, 
                            QLabel, QSlider, QComboBox, QPushButton, QGroupBox, QGridLayout,
                            QDial, QFrame, QSizePolicy, QSpacerItem, QStyle, QStyleFactory)
from PyQt5.QtCore import Qt, QTimer, pyqtSignal, QPropertyAnimation, QPoint, QEasingCurve
from PyQt5.QtGui import QPainter, QColor, QLinearGradient, QPen, QBrush, QFont, QPalette, QIcon
import sounddevice as sd
import threading
import time
from scipy.signal import butter, lfilter
import queue

# Global configuration
sample_rate = 44100  # Sample rate (Hz)
hardware_buffer_size = 1024  # Hardware buffer size
latency = "high"  # Options: "low", "medium", "high"

# Global settings to control effects (default values)
settings = {
    'wave_type': 'sine',     # sine, square, sawtooth, triangle
    'attack': 0.01,          # seconds
    'decay': 0.1,            # seconds
    'sustain': 0.7,          # level (0-1)
    'release': 3.0,          # seconds
    'filter_cutoff': 20000,   # Hz
    'detune': 0.0,           # cents
    'reverb': 0.1,           # 0-1
    'resonance': 1.0,        # Q factor (1.0 = no resonance)
    'volume': 0.8,           # 0-1
    'max_polyphony': 6       # Maximum simultaneous notes
}

# Global control flags and structures
running = True
active_notes = {}  # Dictionary to store currently playing notes
note_lock = threading.Lock()  # For thread-safe note operations
command_queue = queue.Queue()  # Queue for thread-safe communication

# Precalculated waveforms and envelope cache
waveform_cache = {}
env_cache = {}

# The core synthesizer functions remain the same as in your original code
# I'm including the essential ones here and will use the same implementation for the rest

def midi_to_freq(note, cents_offset=0):
    """Convert MIDI note to frequency with optional detuning in cents"""
    return 440 * (2 ** ((note - 69 + cents_offset/100)/12))

def note_name(note_number):
    """Convert MIDI note number to note name (e.g., 60 -> 'C4')"""
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    octave = (note_number // 12) - 1
    note = notes[note_number % 12]
    return f"{note}{octave}"

# Core audio functions - these are the same as in the first implementation
# I'll include a placeholder and you can use the same functions from above

# GUI Components with enhanced styling

class CustomDial(QDial):
    """Custom styled QDial with value indicator"""
    
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setStyleSheet("""
            QDial {
                background-color: #2a2a2a;
                color: #ffffff;
            }
        """)
        self.setNotchesVisible(True)
        self.setFixedSize(80, 80)
    
    def paintEvent(self, event):
        super().paintEvent(event)
        
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        
        # Add a custom dot indicator
        rect = self.rect()
        center = rect.center()
        
        # Calculate position based on current value
        angle = (270 - (self.value() - self.minimum()) * 300 / 
                     (self.maximum() - self.minimum())) % 360
        radius = min(rect.width(), rect.height()) * 0.4
        x = center.x() + radius * np.cos(np.radians(angle))
        y = center.y() - radius * np.sin(np.radians(angle))
        
        # Draw the indicator dot
        painter.setPen(QPen(QColor(0, 160, 255), 3))
        painter.drawEllipse(QPoint(int(x), int(y)), 5, 5)

class CustomSlider(QSlider):
    """Custom styled horizontal slider"""
    
    def __init__(self, orientation=Qt.Horizontal, parent=None):
        super().__init__(orientation, parent)
        self.setStyleSheet("""
            QSlider::groove:horizontal {
                border: 1px solid #999999;
                height: 8px;
                background: #4a4a4a;
                margin: 2px 0;
                border-radius: 4px;
            }
            
            QSlider::handle:horizontal {
                background: qlineargradient(x1:0, y1:0, x2:1, y2:1, stop:0 #0078d7, stop:1 #00a2ff);
                border: 1px solid #00a2ff;
                width: 18px;
                margin: -8px 0;
                border-radius: 9px;
            }
            
            QSlider::handle:horizontal:hover {
                background: qlineargradient(x1:0, y1:0, x2:1, y2:1, stop:0 #00a2ff, stop:1 #0078d7);
            }
        """)

class WaveformDisplay(QWidget):
    """Widget that displays the current waveform shape"""
    
    def __init__(self, parent=None):
        super().__init__(parent)
        self.waveform = "sine"
        self.setMinimumSize(100, 60)
        self.setMaximumHeight(60)
    
    def set_waveform(self, waveform):
        self.waveform = waveform
        self.update()
    
    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        
        # Setup
        width = self.width()
        height = self.height()
        painter.fillRect(0, 0, width, height, QColor(30, 30, 30))
        
        # Draw waveform
        painter.setPen(QPen(QColor(0, 160, 255), 2))
        
        # Generate points for the selected waveform
        points = []
        mid_y = height / 2
        amplitude = height * 0.4
        
        if self.waveform == "sine":
            for x in range(width):
                normalized_x = x / width * 2 * np.pi
                y = mid_y - amplitude * np.sin(normalized_x * 2)
                points.append(QPoint(x, int(y)))
        
        elif self.waveform == "square":
            half_period = width / 4
            for i in range(2):
                # High portion
                start_x = i * width / 2
                end_x = start_x + half_period
                for x in range(int(start_x), int(end_x)):
                    points.append(QPoint(x, int(mid_y - amplitude)))
                
                # Low portion
                start_x = end_x
                end_x = (i + 1) * width / 2
                for x in range(int(start_x), int(end_x)):
                    points.append(QPoint(x, int(mid_y + amplitude)))
        
        elif self.waveform == "sawtooth":
            ramp_width = width / 2
            for i in range(2):
                start_x = i * ramp_width
                for x in range(int(ramp_width)):
                    normalized_x = x / ramp_width
                    y = mid_y - amplitude + normalized_x * amplitude * 2
                    points.append(QPoint(int(start_x + x), int(y)))
        
        elif self.waveform == "triangle":
            quarter_period = width / 8
            for i in range(4):
                # Up ramp
                start_x = i * width / 4
                mid_x = start_x + quarter_period
                for x in range(int(start_x), int(mid_x)):
                    normalized_x = (x - start_x) / quarter_period
                    y = mid_y + amplitude - normalized_x * amplitude * 2
                    points.append(QPoint(int(x), int(y)))
                
                # Down ramp
                end_x = start_x + width / 4
                for x in range(int(mid_x), int(end_x)):
                    normalized_x = (x - mid_x) / quarter_period
                    y = mid_y - amplitude + normalized_x * amplitude * 2
                    points.append(QPoint(int(x), int(y)))
        
        # Draw the points
        if points:
            for i in range(1, len(points)):
                painter.drawLine(points[i-1], points[i])

class ADSRVisualizer(QWidget):
    """Widget that visualizes the ADSR envelope"""
    
    def __init__(self, parent=None):
        super().__init__(parent)
        self.attack = 0.01
        self.decay = 0.1
        self.sustain = 0.7
        self.release = 0.2
        self.setMinimumSize(250, 60)
        self.setMaximumHeight(60)
    
    def set_adsr(self, attack, decay, sustain, release):
        self.attack = attack
        self.decay = decay
        self.sustain = sustain
        self.release = release
        self.update()
    
    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        
        # Setup
        width = self.width()
        height = self.height()
        painter.fillRect(0, 0, width, height, QColor(30, 30, 30))
        
        # Calculate segments
        total_duration = self.attack + self.decay + 0.5 + self.release  # 0.5s for sustain display
        attack_width = (self.attack / total_duration) * width
        decay_width = (self.decay / total_duration) * width
        sustain_width = (0.5 / total_duration) * width
        release_width = (self.release / total_duration) * width
        
        # Draw envelope
        painter.setPen(QPen(QColor(0, 200, 0), 2))
        
        # Attack segment
        start_point = QPoint(0, height)
        attack_peak = QPoint(int(attack_width), 0)
        painter.drawLine(start_point, attack_peak)
        
        # Decay segment
        decay_end = QPoint(int(attack_width + decay_width), int(height * (1 - self.sustain)))
        painter.drawLine(attack_peak, decay_end)
        
        # Sustain segment
        sustain_end = QPoint(int(attack_width + decay_width + sustain_width), int(height * (1 - self.sustain)))
        painter.drawLine(decay_end, sustain_end)
        
        # Release segment
        release_end = QPoint(int(attack_width + decay_width + sustain_width + release_width), height)
        painter.drawLine(sustain_end, release_end)
        
        # Labels
        painter.setPen(Qt.white)
        painter.setFont(QFont("Arial", 8))
        painter.drawText(int(attack_width/2), height-5, "A")
        painter.drawText(int(attack_width + decay_width/2), height-5, "D")
        painter.drawText(int(attack_width + decay_width + sustain_width/2), height-5, "S")
        painter.drawText(int(attack_width + decay_width + sustain_width + release_width/2), height-5, "R")

class PianoKey(QWidget):
    """Widget representing a single piano key with enhanced visuals"""
    clicked = pyqtSignal(int, int)  # Note, Velocity
    released = pyqtSignal(int)  # Note
    
    def __init__(self, note, is_black=False, parent=None):
        super().__init__(parent)
        self.note = note
        self.is_black = is_black
        self.is_pressed = False
        self.is_active = False  # For external activation
        self.setMouseTracking(True)
        
        # Set fixed width based on key type
        if is_black:
            self.setFixedWidth(24)
            self.setFixedHeight(100)
        else:
            self.setFixedWidth(40)
            self.setFixedHeight(150)
    
    def set_active(self, active):
        self.is_active = active
        self.update()
    
    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        
        # Key base color
        if self.is_black:
            base_color = QColor(0, 0, 0)
            pressed_color = QColor(50, 50, 50)
            active_color = QColor(50, 100, 150)
        else:
            base_color = QColor(255, 255, 255)
            pressed_color = QColor(200, 200, 200)
            active_color = QColor(200, 220, 255)
        
        # Select color based on state
        if self.is_pressed:
            color = pressed_color
        elif self.is_active:
            color = active_color
        else:
            color = base_color
        
        # Draw key with gradient
        painter.setPen(Qt.black)
        
        if self.is_black:
            gradient = QLinearGradient(0, 0, 0, self.height())
            gradient.setColorAt(0, color)
            gradient.setColorAt(1, color.darker(150))
            painter.setBrush(QBrush(gradient))
            painter.drawRect(0, 0, self.width(), self.height())
        else:
            gradient = QLinearGradient(0, 0, 0, self.height())
            gradient.setColorAt(0, color)
            gradient.setColorAt(1, color.darker(110))
            painter.setBrush(QBrush(gradient))
            painter.drawRoundedRect(0, 0, self.width(), self.height(), 5, 5)
        
        # Draw note name at bottom of white keys
        if not self.is_black:
            note_name_text = note_name(self.note)
            painter.setPen(Qt.black)
            painter.setFont(QFont("Arial", 8))
            painter.drawText(0, self.height() - 20, self.width(), 20, Qt.AlignCenter, note_name_text)
    
    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.is_pressed = True
            self.clicked.emit(self.note, 100)  # 100 is the velocity (0-127)
            self.update()
    
    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton and self.is_pressed:
            self.is_pressed = False
            self.released.emit(self.note)
            self.update()
    
    def leaveEvent(self, event):
        if self.is_pressed:
            self.is_pressed = False
            self.released.emit(self.note)
            self.update()

class PianoKeyboard(QWidget):
    """Widget displaying an enhanced interactive piano keyboard"""
    
    def __init__(self, start_note=48, num_white_keys=21, parent=None):
        super().__init__(parent)
        self.setMinimumHeight(160)
        self.keys = {}  # Store key widgets by note number
        
        # Create layout
        layout = QHBoxLayout(self)
        layout.setSpacing(0)
        layout.setContentsMargins(0, 0, 0, 0)
        
        # Define note patterns
        is_black_key = [0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0]  # C, C#, D, D#, etc.
        
        # Create white key containers first
        white_key_containers = []
        current_note = start_note
        white_keys_created = 0
        
        while white_keys_created < num_white_keys:
            if not is_black_key[current_note % 12]:
                container = QWidget()
                container_layout = QVBoxLayout(container)
                container_layout.setContentsMargins(0, 0, 0, 0)
                container_layout.setSpacing(0)
                
                white_key = PianoKey(current_note, False)
                white_key.clicked.connect(self.note_on)
                white_key.released.connect(self.note_off)
                
                container_layout.addWidget(white_key)
                white_key_containers.append((container, current_note))
                white_keys_created += 1
                
                # Store key reference
                self.keys[current_note] = white_key
            
            current_note += 1
        
        # Add white keys to layout
        for container, _ in white_key_containers:
            layout.addWidget(container)
        
        # Now add black keys as overlays
        for i, (container, note) in enumerate(white_key_containers[:-1]):
            if is_black_key[(note + 1) % 12]:
                black_key = PianoKey(note + 1, True)
                black_key.clicked.connect(self.note_on)
                black_key.released.connect(self.note_off)
                
                # Position black key over the white keys
                black_key.setParent(self)
                black_key.show()
                black_key.raise_()
                
                # Calculate position (halfway between white keys)
                white_key_width = container.width()
                black_key.move(container.x() + white_key_width - black_key.width() // 2, 0)
                
                # Store key reference
                self.keys[note + 1] = black_key
        
        # Setup timer to update key visuals
        self.update_timer = QTimer()
        self.update_timer.timeout.connect(self.update_key_visuals)
        self.update_timer.start(100)  # Update every 100ms
    
    def resizeEvent(self, event):
        # Reposition black keys when resized
        super().resizeEvent(event)
        
        # Find all black keys
        black_keys = [child for child in self.children() 
                     if isinstance(child, PianoKey) and child.is_black]
        
        # Find all top-level white key containers
        white_containers = [child for child in self.children() 
                           if isinstance(child, QWidget) and not isinstance(child, PianoKey)]
        
        # Reposition each black key
        for i, black_key in enumerate(black_keys):
            # Find the white container before this black key
            if i < len(white_containers):
                container = white_containers[i]
                black_key.move(container.x() + container.width() - black_key.width() // 2, 0)
    
    def note_on(self, note, velocity):
        command_queue.put(('note_on', (note, velocity)))
    
    def note_off(self, note):
        command_queue.put(('note_off', note))
    
    def update_key_visuals(self):
        """Update key visuals based on active notes"""
        with note_lock:
            active_notes_list = list(active_notes.keys())
        
        # Update all keys
        for note, key in self.keys.items():
            key.set_active(note in active_notes_list)

class SettingKnob(QWidget):
    """A knob control with label for synth parameters"""
    valueChanged = pyqtSignal(float)
    
    def __init__(self, label, min_val, max_val, default_val, is_log=False, parent=None):
        super().__init__(parent)
        self.min_val = min_val
        self.max_val = max_val
        self.is_log = is_log
        
        # Main layout
        layout = QVBoxLayout(self)
        layout.setContentsMargins(5, 5, 5, 5)
        
        # Label on top
        self.label = QLabel(label)
        self.label.setAlignment(Qt.AlignCenter)
        self.label.setStyleSheet("font-weight: bold; color: white;")
        
        # Knob
        self.knob = CustomDial()
        self.knob.setRange(0, 100)
        
        # Value display
        self.value_display = QLabel()
        self.value_display.setAlignment(Qt.AlignCenter)
        self.value_display.setStyleSheet("color: #0abdc6; font-family: 'Consolas';")
        
        # Add to layout
        layout.addWidget(self.label)
        layout.addWidget(self.knob, 1)
        layout.addWidget(self.value_display)
        
        # Set initial value
        self.set_value(default_val)
        
        # Connect signals
        self.knob.valueChanged.connect(self._knob_value_changed)
    
    def _knob_value_changed(self, knob_val):
        # Convert from knob range to actual value
        if self.is_log:
            # Log scale mapping
            norm_val = knob_val / 100.0  # Normalize to 0-1
            log_min = np.log10(self.min_val) if self.min_val > 0 else 0
            log_max = np.log10(self.max_val)
            log_val = log_min + norm_val * (log_max - log_min)
            actual_val = 10 ** log_val
        else:
            # Linear scale
            actual_val = self.min_val + (knob_val / 100.0) * (self.max_val - self.min_val)
        
        # Update display
        self._update_display(actual_val)
        
        # Emit signal
        self.valueChanged.emit(actual_val)
    
    def set_value(self, value):
        # Clamp to range
        value = max(self.min_val, min(self.max_val, value))
        
        # Update display
        self._update_display(value)
        
        # Convert to knob value
        if self.is_log:
            # Log scale mapping
            log_min = np.log10(self.min_val) if self.min_val > 0 else 0
            log_max = np.log10(self.max_val)
            log_val = np.log10(value)
            norm_val = (log_val - log_min) / (log_max - log_min)
            knob_val = int(norm_val * 100)
        else:
            # Linear scale
            norm_val = (value - self.min_val) / (self.max_val - self.min_val)
            knob_val = int(norm_val * 100)
        
        # Update knob (will trigger valueChanged)
        self.knob.blockSignals(True)
        self.knob.setValue(knob_val)
        self.knob.blockSignals(False)
    
    def _update_display(self, value):
        # Format display based on value type
        if self.is_log and value >= 1000:
            self.value_display.setText(f"{value/1000:.1f}kHz")
        elif isinstance(value, float):
            self.value_display.setText(f"{value:.2f}")
        else:
            self.value_display.setText(f"{value}")

class SynthGUI(QMainWindow):
    """Enhanced main synthesizer GUI window with visualizations and interactive controls"""
    
    def __init__(self):
        super().__init__()
        self.setWindowTitle('QSynth - Advanced Software Synthesizer')
        self.setGeometry(100, 100, 1200, 800)
        
        # Create central widget and main layout
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget)
        main_layout.setContentsMargins(10, 10, 10, 10)
        main_layout.setSpacing(10)
        
        # Create header with title and visualization section
        header_section = QWidget()
        header_layout = QHBoxLayout(header_section)
        header_layout.setContentsMargins(0, 0, 0, 0)
        
        # Logo/Title area
        title_label = QLabel("QSynth")
        title_label.setStyleSheet("font-size: 28px; font-weight: bold; color: #0abdc6;")
        header_layout.addWidget(title_label)
        
        # Visualization area
        visualizer_group = QGroupBox("Visualization")
        visualizer_layout = QHBoxLayout(visualizer_group)
        
        # Waveform display
        self.waveform_display = WaveformDisplay()
        visualizer_layout.addWidget(self.waveform_display, 1)
        
        # ADSR visualizer
        self.adsr_visualizer = ADSRVisualizer()
        visualizer_layout.addWidget(self.adsr_visualizer, 2)
        
        header_layout.addWidget(visualizer_group, 3)
        
        # Create controls section with knobs
        controls_group = QGroupBox("Synth Controls")
        controls_layout = QGridLayout(controls_group)
        controls_layout.setSpacing(15)
        
        # Waveform selector
        wave_label = QLabel("Waveform:")
        wave_label.setStyleSheet("font-weight: bold;")
        self.wave_combo = QComboBox()
        self.wave_combo.addItems(["sine", "square", "sawtooth", "triangle"])
        self.wave_combo.setCurrentText(settings['wave_type'])
        self.wave_combo.currentTextChanged.connect(
            lambda text: self.update_setting('wave_type', text))
        controls_layout.addWidget(wave_label, 0, 0)
        controls_layout.addWidget(self.wave_combo, 0, 1)
        
        # Create knobs for ADSR
        self.attack_knob = SettingKnob("Attack", 0.001, 2.0, settings['attack'])
        self.attack_knob.valueChanged.connect(
            lambda value: self.update_setting('attack', value))
        controls_layout.addWidget(self.attack_knob, 0, 2)
        
        self.decay_knob = SettingKnob("Decay", 0.001, 2.0, settings['decay'])
        self.decay_knob.valueChanged.connect(
            lambda value: self.update_setting('decay', value))
        controls_layout.addWidget(self.decay_knob, 0, 3)
        
        self.sustain_knob = SettingKnob("Sustain", 0.0, 1.0, settings['sustain'])
        self.sustain_knob.valueChanged.connect(
            lambda value: self.update_setting('sustain', value))
        controls_layout.addWidget(self.sustain_knob, 0, 4)
        
        self.release_knob = SettingKnob("Release", 0.001, 3.0, settings['release'])
        self.release_knob.valueChanged.connect(
            lambda value: self.update_setting('release', value))
        controls_layout.addWidget(self.release_knob, 0, 5)
        
        # Create knobs for other parameters
        self.filter_knob = SettingKnob("Filter Cutoff", 20, 20000, settings['filter_cutoff'], is_log=True)
        self.filter_knob.valueChanged.connect(
            lambda value: self.update_setting('filter_cutoff', value))
        controls_layout.addWidget(self.filter_knob, 1, 2)
        
        self.detune_knob = SettingKnob("Detune", -100, 100, settings['detune'])
        self.detune_knob.valueChanged.connect(
            lambda value: self.update_setting('detune', value))
        controls_layout.addWidget(self.detune_knob, 1, 3)
        
        self.reverb_knob = SettingKnob("Reverb", 0.0, 1.0, settings['reverb'])
        self.reverb_knob.valueChanged.connect(
            lambda value: self.update_setting('reverb', value))
        controls_layout.addWidget(self.reverb_knob, 1, 4)
        
        self.volume_knob = SettingKnob("Volume", 0.0, 1.0, settings['volume'])
        self.volume_knob.valueChanged.connect(
            lambda value: self.update_setting('volume', value))
        controls_layout.addWidget(self.volume_knob, 1, 5)
        
        # Polyphony control
        poly_label = QLabel("Polyphony:")
        poly_label.setStyleSheet("font-weight: bold;")
        self.poly_combo = QComboBox()
        self.poly_combo.addItems([str(i) for i in range(1, 17)])
        self.poly_combo.setCurrentText(str(settings['max_polyphony']))
        self.poly_combo.currentTextChanged.connect(
            lambda text: self.update_setting('max_polyphony', int(text)))
        controls_layout.addWidget(poly_label, 1, 0)
        controls_layout.addWidget(self.poly_combo, 1, 1)
        
        # Add status section
        status_group = QGroupBox("Status")
        status_layout = QHBoxLayout(status_group)
        
        self.active_notes_label = QLabel("Active Notes: 0")
        self.active_notes_label.setStyleSheet("font-family: 'Consolas'; color: #0abdc6;")
        status_layout.addWidget(self.active_notes_label)
        
        self.buffer_label = QLabel(f"Buffer Size: {hardware_buffer_size}")
        self.buffer_label.setStyleSheet("font-family: 'Consolas'; color: #0abdc6;")
        status_layout.addWidget(self.buffer_label)
        
        self.latency_label = QLabel(f"Latency Mode: {latency}")
        self.latency_label.setStyleSheet("font-family: 'Consolas'; color: #0abdc6;")
        status_layout.addWidget(self.latency_label)
        
        # Add piano keyboard section
        keyboard_group = QGroupBox("Keyboard")
        keyboard_layout = QVBoxLayout(keyboard_group)
        
        self.keyboard = PianoKeyboard(48, 21)  # Start at C3, show 21 white keys
        keyboard_layout.addWidget(self.keyboard)
        
        # Add buttons for octave shift
        octave_buttons_layout = QHBoxLayout()
        
        octave_down_btn = QPushButton("◀ Octave Down")
        octave_down_btn.clicked.connect(lambda: self.shift_octave(-12))
        octave_buttons_layout.addWidget(octave_down_btn)
        
        octave_up_btn = QPushButton("Octave Up ▶")
        octave_up_btn.clicked.connect(lambda: self.shift_octave(12))
        octave_buttons_layout.addWidget(octave_up_btn)
        
        keyboard_layout.addLayout(octave_buttons_layout)
        
        # Add all sections to main layout
        main_layout.addWidget(header_section)
        main_layout.addWidget(controls_group)
        main_layout.addWidget(status_group)
        main_layout.addWidget(keyboard_group, 1)  # Give keyboard more vertical space
        
        # Setup update timer
        self.update_timer = QTimer()
        self.update_timer.timeout.connect(self.update_status_and_visuals)
        self.update_timer.start(100)  # Update every 100ms
        
        # Set initial values
        self.keyboard_base_note = 48  # Starting note for keyboard (C3)
        
        # Apply stylesheet
        self.setStyleSheet("""
            QMainWindow, QWidget {
                background-color: #1a1a1a;
                color: #ffffff;
            }
            QGroupBox {
                border: 2px solid #0abdc6;
                border-radius: 8px;
                margin-top: 10px;
                font-weight: bold;
                font-size: 14px;
            }
            QGroupBox::title {
                subcontrol-origin: margin;
                left: 10px;
                padding: 0 5px;
            }
            QLabel {
                color: #e1e1e1;
            }
            QComboBox {
                background-color: #2a2a2a;
                border: 1px solid #0abdc6;
                border-radius: 3px;
                padding: 5px;
                color: white;
            }
            QComboBox::drop-down {
                subcontrol-origin: padding;
                subcontrol-position: top right;
                width: 20px;
                border-left-width: 1px;
                border-left-color: #0abdc6;
                border-left-style: solid;
            }
            QPushButton {
                background-color: #2a2a2a;
                color: white;
                border: 1px solid #0abdc6;
                border-radius: 4px;
                padding: 5px 10px;
            }
            QPushButton:hover {
                background-color: #3a3a3a;
            }
            QPushButton:pressed {
                background-color: #0abdc6;
                color: black;
            }
        """)
    
    def update_setting(self, setting_name, value):
        """Update a synth setting and refresh visuals"""
        command_queue.put(('update_setting', (setting_name, value)))
        
        # Update visualizations
        if setting_name == 'wave_type':
            self.waveform_display.set_waveform(value)
        elif setting_name in ['attack', 'decay', 'sustain', 'release']:
            self.adsr_visualizer.set_adsr(
                settings['attack'],
                settings['decay'],
                settings['sustain'],
                settings['release']
            )
    
    def shift_octave(self, offset):
        """Shift the keyboard up or down by the given offset (in semitones)"""
        self.keyboard_base_note += offset
        # Limit to reasonable MIDI note range
        self.keyboard_base_note = max(12, min(84, self.keyboard_base_note))
        
        # Rebuild the keyboard with the new base note
        keyboard_layout = self.findChild(QGroupBox, "Keyboard").layout()
        old_keyboard = self.keyboard
        keyboard_layout.removeWidget(old_keyboard)
        old_keyboard.deleteLater()
        
        # Create new keyboard with updated base note
        self.keyboard = PianoKeyboard(self.keyboard_base_note, 21)
        keyboard_layout.insertWidget(0, self.keyboard)
    
    def update_status_and_visuals(self):
        """Update status displays and visualizations"""
        # Update active notes count
        with note_lock:
            active_count = len(active_notes)
        self.active_notes_label.setText(f"Active Notes: {active_count}")
        
        # Update ADSR visualization if needed
        self.adsr_visualizer.set_adsr(
            settings['attack'], 
            settings['decay'], 
            settings['sustain'], 
            settings['release']
        )
        
        # Update waveform visualization
        self.waveform_display.set_waveform(settings['wave_type'])
    
    def closeEvent(self, event):
        """Handle window close event"""
        command_queue.put(('shutdown', None))
        # Give the audio thread some time to shut down
        QTimer.singleShot(500, QApplication.quit)
        event.accept()

    # ... existing code ...

# Define the audio callback function
def audio_callback(outdata, frames, time, status):
    """Audio callback function for sounddevice"""
    if status:
        print(status)
    
    # Generate audio data
    with note_lock:
        if not active_notes:
            outdata.fill(0)
            return
        
        # Mix all active notes
        mix = np.zeros(frames)
        for note, note_data in list(active_notes.items()):
            # Generate samples for this note
            samples = generate_samples(note, frames, note_data)
            
            # Add to mix
            mix += samples
            
            # Update note state
            if note_data['state'] == 'release' and note_data['release_time'] >= settings['release']:
                # Note is finished
                del active_notes[note]
    
    # Apply master volume
    mix *= settings['volume']
    
    # Prevent clipping
    if np.max(np.abs(mix)) > 0.99:
        mix = mix / np.max(np.abs(mix)) * 0.99
    
    # Output stereo
    outdata[:] = np.column_stack((mix, mix))

def generate_samples(note, num_samples, note_data):
    """Generate audio samples for a note"""
    # Get frequency with detune
    freq = midi_to_freq(note, settings['detune'])
    
    # Get current phase
    phase = note_data['phase']
    
    # Generate raw waveform
    if settings['wave_type'] == 'sine':
        samples = np.sin(2 * np.pi * freq * np.arange(num_samples) / sample_rate + phase)
    elif settings['wave_type'] == 'square':
        samples = np.sign(np.sin(2 * np.pi * freq * np.arange(num_samples) / sample_rate + phase))
    elif settings['wave_type'] == 'sawtooth':
        t = (freq * np.arange(num_samples) / sample_rate + phase/2/np.pi) % 1
        samples = 2 * t - 1
    elif settings['wave_type'] == 'triangle':
        t = (freq * np.arange(num_samples) / sample_rate + phase/2/np.pi) % 1
        samples = 2 * np.abs(2 * t - 1) - 1
    
    # Update phase for next call
    note_data['phase'] = (phase + 2 * np.pi * freq * num_samples / sample_rate) % (2 * np.pi)
    
    # Apply envelope
    if note_data['state'] == 'attack':
        # Attack phase
        attack_samples = int(settings['attack'] * sample_rate)
        if note_data['time'] + num_samples < attack_samples:
            # Still in attack
            start_level = note_data['time'] / attack_samples
            end_level = (note_data['time'] + num_samples) / attack_samples
            envelope = np.linspace(start_level, end_level, num_samples)
            note_data['time'] += num_samples
        else:
            # Transition to decay
            samples_in_attack = max(0, attack_samples - note_data['time'])
            if samples_in_attack > 0:
                start_level = note_data['time'] / attack_samples
                envelope_attack = np.linspace(start_level, 1.0, samples_in_attack)
            else:
                envelope_attack = np.array([])
            
            # Start decay phase
            samples_in_decay = num_samples - samples_in_attack
            if samples_in_decay > 0:
                decay_samples = int(settings['decay'] * sample_rate)
                end_level = max(settings['sustain'], 1.0 - samples_in_decay / decay_samples)
                envelope_decay = np.linspace(1.0, end_level, samples_in_decay)
            else:
                envelope_decay = np.array([])
            
            envelope = np.concatenate((envelope_attack, envelope_decay))
            note_data['state'] = 'decay'
            note_data['time'] = samples_in_decay
    
    elif note_data['state'] == 'decay':
        # Decay phase
        decay_samples = int(settings['decay'] * sample_rate)
        if note_data['time'] + num_samples < decay_samples:
            # Still in decay
            start_level = 1.0 - note_data['time'] / decay_samples * (1.0 - settings['sustain'])
            end_level = 1.0 - (note_data['time'] + num_samples) / decay_samples * (1.0 - settings['sustain'])
            envelope = np.linspace(start_level, end_level, num_samples)
            note_data['time'] += num_samples
        else:
            # Transition to sustain
            samples_in_decay = max(0, decay_samples - note_data['time'])
            if samples_in_decay > 0:
                start_level = 1.0 - note_data['time'] / decay_samples * (1.0 - settings['sustain'])
                envelope_decay = np.linspace(start_level, settings['sustain'], samples_in_decay)
            else:
                envelope_decay = np.array([])
            
            # Start sustain phase
            samples_in_sustain = num_samples - samples_in_decay
            if samples_in_sustain > 0:
                envelope_sustain = np.ones(samples_in_sustain) * settings['sustain']
            else:
                envelope_sustain = np.array([])
            
            envelope = np.concatenate((envelope_decay, envelope_sustain))
            note_data['state'] = 'sustain'
            note_data['time'] = 0  # Reset time for sustain
    
    elif note_data['state'] == 'sustain':
        # Sustain phase (constant level)
        envelope = np.ones(num_samples) * settings['sustain']
    
    elif note_data['state'] == 'release':
        # Release phase
        release_samples = int(settings['release'] * sample_rate)
        start_level = settings['sustain'] * (1.0 - note_data['release_time'] / settings['release'])
        note_data['release_time'] += num_samples / sample_rate
        end_level = settings['sustain'] * (1.0 - note_data['release_time'] / settings['release'])
        end_level = max(0, end_level)  # Ensure non-negative
        envelope = np.linspace(start_level, end_level, num_samples)
    
    # Apply envelope
    samples = samples * envelope
    
    # Apply simple lowpass filter (if cutoff is below Nyquist)
    if settings['filter_cutoff'] < sample_rate / 2:
        # Simple one-pole lowpass filter
        for i in range(1, len(samples)):
            alpha = 0.1  # Filter coefficient (lower = more filtering)
            samples[i] = alpha * samples[i] + (1 - alpha) * samples[i-1]
    
    return samples

# Function to process commands from the GUI
def command_processor():
    global running, settings
    
    while running:
        try:
            command, data = command_queue.get(timeout=0.1)
            
            if command == 'note_on':
                note, velocity = data
                with note_lock:
                    # Check polyphony limit
                    if len(active_notes) >= settings['max_polyphony'] and note not in active_notes:
                        # Find the oldest note to replace
                        oldest_note = min(active_notes.keys(), 
                                         key=lambda n: active_notes[n].get('start_time', 0))
                        del active_notes[oldest_note]
                    
                    # Add or retrigger the note
                    active_notes[note] = {
                        'velocity': velocity / 127.0,
                        'phase': 0,
                        'state': 'attack',
                        'time': 0,
                        'release_time': 0,
                        'start_time': time.time()
                    }
            
            elif command == 'note_off':
                note = data
                with note_lock:
                    if note in active_notes:
                        active_notes[note]['state'] = 'release'
                        active_notes[note]['release_time'] = 0
            
            elif command == 'update_setting':
                setting_name, value = data
                settings[setting_name] = value
            
            elif command == 'shutdown':
                running = False
                break
        
        except queue.Empty:
            pass
        except Exception as e:
            print(f"Error in command processor: {e}")

# Start the audio stream and GUI
if __name__ == "__main__" or 'get_ipython' in globals():
    # Start audio stream
    try:
        stream = sd.OutputStream(
            samplerate=sample_rate,
            blocksize=hardware_buffer_size,
            channels=2,
            callback=audio_callback,
            latency=latency
        )
        stream.start()
        
        # Start command processor thread
        cmd_thread = threading.Thread(target=command_processor)
        cmd_thread.daemon = True
        cmd_thread.start()
        
        # Start GUI
        app = QApplication(sys.argv)
        gui = SynthGUI()
        gui.show()
        app.exec_()
        
        # Cleanup
        running = False
        if cmd_thread.is_alive():
            cmd_thread.join(timeout=1.0)
        stream.stop()
        stream.close()
        
    except Exception as e:
        print(f"Error starting synth: {e}")

output underflow
output underflow


KeyboardInterrupt: 